In [1]:
import json, pprint, requests, textwrap
host = 'http://livyserveraddress'
data = {'kind': 'pyspark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 117,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [2]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 117,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [4]:
#codefile = open('C:\livy_files\load_libs_dataframes_livy.py','r')
#code = codefile.read()
#codefile.close()
data = {
        'kind' : 'pyspark',#'code' : textwrap.dedent(code)
        'code' : textwrap.dedent("""\
       
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc,asc
from pyspark.sql.functions import lower, col
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import from_unixtime, hour, dayofyear
from pyspark.sql import Window
  
df_movies = (spark.read\
            .format('csv')
            .option('header','true')
            .option('delimiter',",")
            .option('inferSchema','true')
            .load("/home/administrator/Downloads/movielens/movie.csv"))





df_rating = (spark.read\
            .format('csv')
            .option('header','true')
            .option('delimiter',",")
            .option('inferSchema','true')
            .load("/home/administrator/Downloads/movielens/rating.csv"))





df_tag = (spark.read\
         .format('csv')
         .option('header','true')
         .option('delimiter',",")
         .option('inferSchema','true')
         .load("/home/administrator/Downloads/movielens/tag.csv"))





df_genome_tags = (spark.read\
                 .format('csv')
                 .option('header','true')
                 .option('delimiter',",")
                 .option('inferSchema','true')
                 .load("/home/administrator/Downloads/movielens/genome_tags.csv"))
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 0,
 'code': '\nfrom pyspark.sql import SparkSession\nfrom pyspark.sql.functions import desc,asc\nfrom pyspark.sql.functions import lower, col\nfrom pyspark.sql.types import *\nfrom pyspark.sql.functions import *\nfrom pyspark.sql.functions import from_unixtime, hour, dayofyear\nfrom pyspark.sql import Window\n\ndf_movies = (spark.read            .format(\'csv\')\n            .option(\'header\',\'true\')\n            .option(\'delimiter\',",")\n            .option(\'inferSchema\',\'true\')\n            .load("/home/administrator/Downloads/movielens/movie.csv"))\n\n\n\n\n\ndf_rating = (spark.read            .format(\'csv\')\n            .option(\'header\',\'true\')\n            .option(\'delimiter\',",")\n            .option(\'inferSchema\',\'true\')\n            .load("/home/administrator/Downloads/movielens/rating.csv"))\n\n\n\n\n\ndf_tag = (spark.read         .format(\'csv\')\n         .option(\'header\',\'true\')\n         .option(\'delimiter\',",")\n         .option(\'inferSc

In [5]:
#query 1 executio,find jumanji viewers
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
       
joinedq1 = df_movies.filter(df_movies.title.like("%Jumanji%")).join(df_rating, df_rating["movieId"] == df_movies["movieId"], "inner").drop(df_rating.movieId).select(df_rating['userId'],df_movies["title"]).groupBy(df_rating.userId).count().agg(sum(col('count')))
joinedq1.select(col('sum(count)').alias('Jumanji_viewers')).show()
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 1,
 'code': '\njoinedq1 = df_movies.filter(df_movies.title.like("%Jumanji%")).join(df_rating, df_rating["movieId"] == df_movies["movieId"], "inner").drop(df_rating.movieId).select(df_rating[\'userId\'],df_movies["title"]).groupBy(df_rating.userId).count().agg(sum(col(\'count\')))\njoinedq1.select(col(\'sum(count)\').alias(\'Jumanji_viewers\')).show()\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [6]:
#query 2 
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
df_low=df_tag.withColumn("tag", lower(col("tag")))      
df_low_clr = df_low.select(col('userId'),col('movieId'),regexp_replace(df_low.tag,'[^a-zA-Z0-9]',"").alias('tag_clr'))
joinq2 = df_movies.join(df_low_clr,df_movies['movieId'] == df_low_clr['movieId'],'inner').drop(df_movies.movieId).filter(df_low_clr.tag_clr.like('boring')).dropDuplicates(subset=['movieId']).select(col('userId'),col('movieId'),col('title'),col('tag_clr').alias('tag')).orderBy(col('title').asc()).show(truncate=False)
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 2,
 'code': 'df_low=df_tag.withColumn("tag", lower(col("tag")))      \ndf_low_clr = df_low.select(col(\'userId\'),col(\'movieId\'),regexp_replace(df_low.tag,\'[^a-zA-Z0-9]\',"").alias(\'tag_clr\'))\njoinq2 = df_movies.join(df_low_clr,df_movies[\'movieId\'] == df_low_clr[\'movieId\'],\'inner\').drop(df_movies.movieId).filter(df_low_clr.tag_clr.like(\'boring\')).dropDuplicates(subset=[\'movieId\']).select(col(\'userId\'),col(\'movieId\'),col(\'title\'),col(\'tag_clr\').alias(\'tag\')).orderBy(col(\'title\').asc()).show(truncate=False)\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [7]:
#query 3
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
df3_ratings = df_rating.filter(df_rating.rating > 3).select(df_rating.userId,df_rating.rating)
df_tagboll = df_low_clr.filter(df_low_clr.tag_clr.like('bollywood')).select(df_low_clr.movieId,df_low_clr.userId,df_low_clr.tag_clr.alias('tag'))

join3_ratag = (df3_ratings.join(df_tagboll,df3_ratings['userId'] == df_tagboll['userId'],"inner")\
                           .drop(df3_ratings.userId).select(df_tagboll.movieId,df_tagboll['userId'].alias("userIdJoined"),df_tagboll['tag'],df_rating['rating'])).dropDuplicates(subset = ['userIdJoined']).orderBy(col('userIdJoined').asc()).show(truncate=False)
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 3,
 'code': 'df3_ratings = df_rating.filter(df_rating.rating > 3).select(df_rating.userId,df_rating.rating)\ndf_tagboll = df_low_clr.filter(df_low_clr.tag_clr.like(\'bollywood\')).select(df_low_clr.movieId,df_low_clr.userId,df_low_clr.tag_clr.alias(\'tag\'))\n\njoin3_ratag = (df3_ratings.join(df_tagboll,df3_ratings[\'userId\'] == df_tagboll[\'userId\'],"inner")                           .drop(df3_ratings.userId).select(df_tagboll.movieId,df_tagboll[\'userId\'].alias("userIdJoined"),df_tagboll[\'tag\'],df_rating[\'rating\'])).dropDuplicates(subset = [\'userIdJoined\']).orderBy(col(\'userIdJoined\').asc()).show(truncate=False)\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [8]:
#query 4
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
rating_years = df_rating.select(df_rating.movieId,year(df_rating.timestamp).alias('rate_year'),df_rating.rating)
rating_years_q = rating_years.join(df_movies,rating_years['movieId'] == df_movies['movieId'],'inner').drop(df_movies.movieId).groupBy(col('title'),col('rate_year')).avg('rating').withColumn("rating_rank",row_number().over(Window.partitionBy("rate_year").orderBy(col('avg(rating)').desc()))).orderBy(col('rate_year').asc()).filter(col('rating_rank')<=10).withColumn("alphab_rank",row_number().over(Window.partitionBy("rate_year").orderBy(col('title').asc())))#.dropDuplicates(subset=['title'])
rating_years_q.show(truncate=False)
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 4,
 'code': 'rating_years = df_rating.select(df_rating.movieId,year(df_rating.timestamp).alias(\'rate_year\'),df_rating.rating)\nrating_years_q = rating_years.join(df_movies,rating_years[\'movieId\'] == df_movies[\'movieId\'],\'inner\').drop(df_movies.movieId).groupBy(col(\'title\'),col(\'rate_year\')).avg(\'rating\').withColumn("rating_rank",row_number().over(Window.partitionBy("rate_year").orderBy(col(\'avg(rating)\').desc()))).orderBy(col(\'rate_year\').asc()).filter(col(\'rating_rank\')<=10).withColumn("alphab_rank",row_number().over(Window.partitionBy("rate_year").orderBy(col(\'title\').asc())))#.dropDuplicates(subset=[\'title\'])\nrating_years_q.show(truncate=False)\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [9]:
#query 5
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
df_join5 =  df_movies.join(df_tag,df_movies['movieId'] == df_tag['movieId'],'inner').select(df_tag['tag'],df_movies['title'],df_tag.timestamp)
df_join5_conc = df_join5.filter(year(df_join5.timestamp)==2015).groupBy(col("title")).agg(collect_list(col("tag")).alias('tag')).withColumn("tag", concat_ws(",", col("tag"))).orderBy(col('title').asc()).show(truncate=False)
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 5,
 'code': 'df_join5 =  df_movies.join(df_tag,df_movies[\'movieId\'] == df_tag[\'movieId\'],\'inner\').select(df_tag[\'tag\'],df_movies[\'title\'],df_tag.timestamp)\ndf_join5_conc = df_join5.filter(year(df_join5.timestamp)==2015).groupBy(col("title")).agg(collect_list(col("tag")).alias(\'tag\')).withColumn("tag", concat_ws(",", col("tag"))).orderBy(col(\'title\').asc()).show(truncate=False)\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [10]:
#query 6
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
join6q = df_movies.join(df_rating, df_movies['movieId'] == df_rating['movieId'],"inner").drop(df_rating.movieId).groupBy(df_movies.title).count().orderBy(col("count").desc()).show(truncate=False)
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 6,
 'code': 'join6q = df_movies.join(df_rating, df_movies[\'movieId\'] == df_rating[\'movieId\'],"inner").drop(df_rating.movieId).groupBy(df_movies.title).count().orderBy(col("count").desc()).show(truncate=False)\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [11]:
#query 7
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
count_ratings = df_rating.groupBy(col('userId'),year(df_rating.timestamp)).count().select(col('userId'),col('year(timestamp)').alias('rating_year'),col('count').cast('int').alias('rating_count')).withColumn('count_rank',row_number().over(Window.partitionBy('rating_year').orderBy(col('rating_count').desc()))).orderBy(col('rating_year').asc()).filter(col('count_rank')<=10).withColumn('id_alphab_rank',row_number().over(Window.partitionBy('rating_year').orderBy(col('userId').asc())))
count_ratings.show(truncate=False)
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 7,
 'code': "count_ratings = df_rating.groupBy(col('userId'),year(df_rating.timestamp)).count().select(col('userId'),col('year(timestamp)').alias('rating_year'),col('count').cast('int').alias('rating_count')).withColumn('count_rank',row_number().over(Window.partitionBy('rating_year').orderBy(col('rating_count').desc()))).orderBy(col('rating_year').asc()).filter(col('count_rank')<=10).withColumn('id_alphab_rank',row_number().over(Window.partitionBy('rating_year').orderBy(col('userId').asc())))\ncount_ratings.show(truncate=False)\n",
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [12]:
#query 8
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
movies_genres = df_movies.withColumn("genres",explode(split("genres","[|]")))
genres_nodupes = movies_genres.dropDuplicates(subset=['movieId']).filter(~col('genres').like('%(no genres listed)%')).orderBy(col('movieId').asc())
rating_group = df_rating.groupBy(df_rating.movieId).count()

quer8_join = genres_nodupes.join(rating_group,rating_group['movieId'] == genres_nodupes['movieId'],"inner").drop(genres_nodupes.movieId).dropDuplicates(subset=['movieId']).withColumn("count_rtng_rank",row_number().over(Window.partitionBy("genres").orderBy(col('count').desc()))).filter(col('count_rtng_rank')==1).orderBy(col("genres").asc())
quer8_join.show(truncate=False)
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 8,
 'code': 'movies_genres = df_movies.withColumn("genres",explode(split("genres","[|]")))\ngenres_nodupes = movies_genres.dropDuplicates(subset=[\'movieId\']).filter(~col(\'genres\').like(\'%(no genres listed)%\')).orderBy(col(\'movieId\').asc())\nrating_group = df_rating.groupBy(df_rating.movieId).count()\n\nquer8_join = genres_nodupes.join(rating_group,rating_group[\'movieId\'] == genres_nodupes[\'movieId\'],"inner").drop(genres_nodupes.movieId).dropDuplicates(subset=[\'movieId\']).withColumn("count_rtng_rank",row_number().over(Window.partitionBy("genres").orderBy(col(\'count\').desc()))).filter(col(\'count_rtng_rank\')==1).orderBy(col("genres").asc())\nquer8_join.show(truncate=False)\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [13]:
#query 9
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
df_hour_day = df_rating.select(col('movieId'),col('userId'),regexp_extract(df_rating.timestamp,'\d\d\d\d\-\\d\d\-\\d\d\\s\d\d',0).alias('day_hour')).groupBy(col('movieId'),col('day_hour')).count().filter(col('count')>1).agg(sum(col('count'))).show()
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 9,
 'code': "df_hour_day = df_rating.select(col('movieId'),col('userId'),regexp_extract(df_rating.timestamp,'\\d\\d\\d\\d\\-\\d\\d\\-\\d\\d\\s\\d\\d',0).alias('day_hour')).groupBy(col('movieId'),col('day_hour')).count().filter(col('count')>1).agg(sum(col('count'))).show()\n",
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [14]:
#query 10
data = {
        'kind' : 'pyspark',
        'code' : textwrap.dedent("""\
join_movies10 = df_low_clr.join(genres_nodupes,df_low_clr['movieId']  == genres_nodupes['movieId']).drop(genres_nodupes.movieId)
df_q10 = df_rating.join(join_movies10,df_rating['movieId'] == join_movies10['movieId'],'inner').drop(df_rating.movieId).filter(col('rating')>3.5).filter(join_movies10.tag_clr.like("funny")).dropDuplicates(subset=['movieId']).groupBy(col('genres')).count().orderBy(col('genres').asc())
df_q10.show(truncate=False)
        """)   
}


#session_url = host + '/sessions/110'#instead of the last number here in the /sessions/... u can put ur session id to have the correct connection


statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 10,
 'code': 'join_movies10 = df_low_clr.join(genres_nodupes,df_low_clr[\'movieId\']  == genres_nodupes[\'movieId\']).drop(genres_nodupes.movieId)\ndf_q10 = df_rating.join(join_movies10,df_rating[\'movieId\'] == join_movies10[\'movieId\'],\'inner\').drop(df_rating.movieId).filter(col(\'rating\')>3.5).filter(join_movies10.tag_clr.like("funny")).dropDuplicates(subset=[\'movieId\']).groupBy(col(\'genres\')).count().orderBy(col(\'genres\').asc())\ndf_q10.show(truncate=False)\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [17]:
r = requests.delete(session_url, headers=headers)#close livy session
pprint.pprint(r.json())